# Regression

Im Gegensatz zu einer Klassifikation, die *diskrete Werte* vorhersagt, hat die Regression *kontinuierliche Werte* als Ergebnis.

Bei der *linearen Regression* geht es darum eine *Gerade* zu finden, die die Zielwerte möglichst exakt aus den unabhängigen Variablen vorhersagt. Die *Regressionsgerade* wird normalerweise durch eine Gleichung dargestellt ("parametrisiert"):

$$ y = mx + t $$

$m$ ist dabei die Steigung der Geraden, $t$ der sog. *Achsenabschnitt*.
Solltest du mehrere unabhängige Variablen haben (das wirst du später genauer betrachten), kannst du mit Vektoren rechnen:

$$ y = {\mathbf m} {\mathbf x} + t $$

Wir fangen zunächst ganz einfach an. Auch für die Regression gibt es Datensets, z.B. das der Hauspreise in Kalifornien (das vielbenutzte Boston-Datenset sollte man nicht mehr verwenden!). Das wirst du nun zunächst einladen und anschließend analysieren.

## Datenset laden

Das Prozedere kennst du schon, allerdings ist das Datenset nicht ganz so eng in `sklearn` integriert, daher ist der Aufruf ein bisschen anders:

In [ ]:
from sklearn import datasets
california = datasets.fetch_california_housing()

In [ ]:
print(california.DESCR)

Wie gehabt wandelst du das Datenset in einen `DataFrame`.

In [ ]:
import pandas as pd
df = pd.DataFrame(california.data, columns=california.feature_names)
df["Price"] = california.target
df

## Korrelationen

Zunächst interessierst du dich für die Korrelationen der einzelnen Spalten:

In [ ]:
df.corr()

Das sid sog. *Pearson-R-Werte*. Werte um `0` drücken keine Korrelation aus, `1` ist perfekt korreliert, `-1` ist perfekt antikorreliert. 

Offenbar ist das mittlere Einkommen `MedInc` sehr gut mit dem `Price` korreliert. Das wirst du gleich ausnutzen. Schau dir zunächst aber nochmal die Verteilung an:

In [ ]:
df.plot.scatter(x="MedInc", y="Price", c="#0044aa44", figsize=(16,9))

Die Hauspreise sind bei `5` (entspricht 500.000\$) abgeschnitten, das ist natürlich nicht so günstig! Wenn du möchtest, kannst du die mit `df5 = df[df["Price"] < 5]` filtern, in diesem Beispiel lassen wir die mit drin.

## Einfache lineare Regression

Wie schon von der Klassifikation gewohnt nennst du die unabhängige Variable `X`, die abhängige `y`:

In [ ]:
X = df[["MedInc"]]
y = df["Price"]

Die lineare Regression kannst du mit  `sklearn` ganz einfach durchführen:

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X, y)

Als Ergebnis erhältst du das oben angesprochene $m$ (die Steigung), das hier als `coef_` bezeichnet wird

In [ ]:
lr.coef_

Der Kaufpreis wächst also "nur" mit 40% des Median-Einkommens.

Auch den Achsenabschnitt kannst du ermitteln:

In [ ]:
lr.intercept_

Ohne Einkommen zahlen die Menschen im Mittel also 45.000\$ für ein Haus.

Nun könntest du die Vorhersage mit der Geradengleichung berechnen, oder du nutzt dafür die `predict`-Methode:

In [ ]:
df["Predicted Price"] = lr.predict(df[["MedInc"]])

Jetzt kannst du die Vorhersage mit den echten Werten zusammen einzeichnen.

In [ ]:
df.plot.scatter(x="MedInc", y="Price", c="#0044aa44", figsize=(16, 9));
df.set_index("MedInc")["Predicted Price"].plot(figsize=(16, 9));

Das sieht schon ganz gut aus, aber wie kannst du jetzt den Fehler berechnen?

## Fehler berechnen

Dazu gibt es mehrere Ansätze. Du kannst die *mittlere quadratische Abweichung* berechnen. Quadratiert wird die Abweichung, damit sich positive und negative Abweichungen nicht gegeneinander aufheben.

In [ ]:
from sklearn import metrics
metrics.mean_squared_error(y, df["Predicted Price"])

Eine andere Möglichkeit ist es, den mittleren absoluten Fehler zu berechnen (statt zu quadrieren nutzt du den Betrag):

In [ ]:
metrics.mean_absolute_error(y, df["Predicted Price"])

Wo hat die Vorhersagen den größten Fehler gemacht?

In [ ]:
metrics.max_error(y, df["Predicted Price"])

Die Fehler sind abhängig von der Größe der vorhergesagten Werte. Möchtest du eine davon unabhängige Größe berechnen, kannst du den `r2`-Wert verwenden:

In [ ]:
metrics.r2_score(y, df["Predicted Price"])

Je näher der `r2`-Wert bei `1` liegt, desto besser ist der Fit.

## Zusammenfassung

Regression funktioniert nicht so viel anders als Klassifikation. Allerdings gibt es hier keine *Confusion Matrix*, *Precision* und *Recall*, sondern der Fehler muss anders berechnet werden.

Bisher sollte der Regressor nur bekannte Werte vorhersagen - er könnte als auch auswendig gelernt haben. Genau wie bei der Klassifikation kannst du auch bei der Regresion mit einem Trainings- und davon unabhängigen Test-Datenset arbeiten.